<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/snli_figure2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/NLP/imarg"

Mounted at /content/gdrive


In [5]:
%%capture
import sys
sys.path.append("input-marginalization")

from metrics import input_marginalization, colored_sentence
import torch
from transformers import BertForMaskedLM

In [6]:
%%capture
mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()

In [39]:
#bert = torch.load(f"{save_dir}/bert_sst2.pt")
#cnn = torch.load(f"{save_dir}/cnn_sst2.pt")
lstm = torch.load(f"{save_dir}/lstm_snli.pt")

In [13]:
lstm_sentences = [
  ("pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .", 1),
  ("an important movie , a reminder of the power of file to move us and to make us examine our values .", 1),
  ("unflinchingly bleak and desperate", 0),
  ("i am sorry that i was unable to get the full brunt of this comedy .", 0),
]
bert_sentences = [
  ("it ' s a lovely film with lovely performances by buy and accorsi .", 1),
  ("more romantic , emotional and ultimately more satisfying than the teary - eyed original .", 1),
  ("it ' s a bit disappointing that it only manages to be decent instead of dead brilliant .", 0),
  ("suffers from the lack of a compelling or comprehensible narrative .", 0),   
]
cnn_sentences = [
  (". . . very funny , very enjoyable . . .", 1),
  ("lookin ' for sin , american - style ?", 0),
]

snli_lstm_sentences = [
  (('Two women are embracing while holding to go packages.', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.'), 2),
]

In [64]:
for sentences, label in snli_lstm_sentences:
  att_scores = input_marginalization(lstm, sentences, mlm, target_label=label, dataset='snli')
  print(att_scores)
  #colored_sentence(sentence, att_scores)

tensor([[0.7161, 0.8776, 0.7331, 0.7319, 0.7362, 0.7376, 0.7327, 0.7326, 0.7327,
         0.7149, 0.7327, 0.7551, 0.7326, 0.7454, 0.7341, 0.7437, 0.7309, 0.9402,
         0.7326, 0.7327, 0.7326, 0.7361, 0.7327, 0.7332, 0.7190, 0.7317, 0.7327,
         0.6560]])


In [35]:
# try:
#   for sentence, label in bert_sentences:
#     att_scores = input_marginalization(bert, sentence, mlm, target_label=label)
#     color_sentence(sentence, att_scores)
# except KeyboardInterrupt:
#   print("Graceful Exit")

In [ ]:
# for sentence, label in cnn_sentences:
#   att_scores = input_marginalization(cnn, sentence, mlm, target_label=label)
#   color_sentence(sentence, att_scores)